In [11]:
import jax

import jax.numpy as jnp
from jax import grad,hessian, jit, vmap, partial

from multiinstance.data.realData import buildDataset as buildReal
from glob import glob

from jax.test_util import check_grads

import numpy as np
import matplotlib.pyplot as plt

from multiinstance.data.syntheticData import buildDataset

from multiinstance.ward_clustering import WardClustering

from multiinstance.utils import *

import scipy.stats as ss
from tqdm.notebook import tqdm

from multiinstance.data.realData import buildDataset as buildReal
from glob import glob

In [21]:
def logLikelihood(alphas, mu, sigma):
    ll = jnp.sum(jax.scipy.stats.norm.logpdf(alphas,mu,sigma))
    return ll / len(alphas)


def getLevelClusters(rowNum, clusterAssignments):
    clusterLabels = jnp.unique(clusterAssignments[rowNum])
    clusters = {c : jnp.where(clusterAssignments[rowNum] == c)[0] for c in clusterLabels}
    return clusters

def getClusterMean(leafMeans, numU, clusterMembers):
    n = numU[clusterMembers]
    a = leafMeans[clusterMembers]
    return jnp.dot(a,n) * (1 / jnp.sum(n))
    
def treeNegLogLikelihood(leafMeans, clusterVars, alphaHatMat, numU, clusterAssigments, loc2Idx):
    NLL = 0
    for rowNum in range(clusterAssigments.shape[0]):
        clusters = getLevelClusters(rowNum, clusterAssigments)
        for clusterIdx, clusterMembers in clusters.items():
            clusterMean = getClusterMean(leafMeans, numU, clusterMembers)
            varIdx = loc2Idx[(rowNum, clusterIdx)]
            clusterVar = clusterVars[varIdx]
            alphaHats = alphaHatMat[rowNum, clusterIdx]
            NLL = NLL - logLikelihood(alphaHats, clusterMean, clusterVar)
    return NLL

In [22]:
def prep(alphaHatMat, clusterAssignments):
    leafMeans = jnp.mean(alphaHatMat[0],axis=1)
    clusterVars = []
    loc2Idx = {}
    for rowNum in range(clusterAssignments.shape[0]):
        levelClusters = jnp.unique(clusterAssignments[rowNum])
        for cluster in levelClusters:
            loc2Idx[(rowNum,cluster)] = len(clusterVars)
            alphaHats = alphaHatMat[rowNum, cluster]
            _,v = ss.norm.fit(alphaHats)
            clusterVars.append(v)
    clusterVars = jnp.array(clusterVars)
    return leafMeans, clusterVars, loc2Idx

In [26]:
def run(ds, clusterAssignments, alphaHatMat, NIterations=100, lr=0.01):
    
    leafMeans, clusterVars, loc2Idx = prep(alphaHatMat, clusterAssignments)
#     meanHistory = [leafMeans]
#     scaleHistory = [clusterVars]
#     maes = [jnp.mean(jnp.abs(ds.trueAlphas.flatten() - leafMeans))]
    numU = jnp.array(ds.numU)
    # Define gradient and hessian
    meanVarGrad = grad(treeNegLogLikelihood,argnums=(0,1))
    meanHessian = jax.jacfwd(jax.jacrev(lambda m,v: treeNegLogLikelihood(m, v, alphaHatMat, numU, clusterAssignments, loc2Idx)))
    varHessian = jax.jacfwd(jax.jacrev(lambda v,m: treeNegLogLikelihood(m, v, alphaHatMat, numU, clusterAssignments, loc2Idx)))
    # Run Iterations
    for iteration in tqdm(range(NIterations),total=NIterations,leave=False):
        meanGrad, varGrad = meanVarGrad(leafMeans, clusterVars, alphaHatMat, numU, clusterAssignments, loc2Idx)
        leafMeans = leafMeans - lr * jnp.linalg.inv(meanHessian(leafMeans, clusterVars)) @ meanGrad
        clusterVars = clusterVars - lr * jnp.linalg.inv(varHessian(clusterVars, leafMeans)) @ varGrad
#         meanHistory.append(leafMeans)
#         scaleHistory.append(clusterVars)
#         maes.append(jnp.mean(jnp.abs(ds.trueAlphas.flatten() - leafMeans)))
#     return leafMeans, clusterVars, loc2Idx, meanHistory, scaleHistory, maes
    return leafMeans

In [24]:
from sklearn.metrics import roc_auc_score
def posteriorCorrection(tau, alpha, S0S1):
    post =  alpha * S0S1 * (tau / (1 - tau))
    post[np.isinf(post)] = 1
    return post

def correctedAUC(ds,bagAlphaHats,):
    _, tauArrays = list(zip(*[getTransformScores(ds,i) for i in range(ds.N)]))
    S0_S1 = ds.numU/ds.numP
    posteriors = [posteriorCorrection(tau,alphaHat, s0s1) for tau,alphaHat,s0s1 in zip(tauArrays,
                                                                                       bagAlphaHats,
                                                                                       S0_S1)]
    posteriorVals = np.concatenate(posteriors)
    hiddenLabels = np.concatenate([ds.hiddenLabels[i][:ds.numU[i]] for i in range(ds.N)])
    return roc_auc_score(hiddenLabels, posteriorVals)

In [28]:
absErrs = {"local":[],
           "forest":[],
           "global": []}

aucVals = {"local":[],
           "forest":[],
           "global": []}
N = 0
NIters = 5
for f in tqdm(glob("/ssdata/ClassPriorEstimationPrivate/data/rawDatasets/*.mat")):
# for f in tqdm(glob("/data/dzeiberg/ClassPriorEstimation/rawDatasets/*.mat")):
    dsi = buildReal(f,16,
                    alphaDistr=lambda: np.random.uniform(.05,.95),
                    nPDistr=lambda: 1 + np.random.poisson(10),
                    nUDistr=lambda: 1 + np.random.poisson(25))
    dsi = addTransformScores(dsi)
    dsi = addGlobalEsts(dsi,reps=10)
    dsi.alphaHats,dsi.curves = getBagAlphaHats(dsi,
                                               numbootstraps=100)
    globalAE = np.abs(dsi.trueAlphas.flatten() - dsi.globalAlphaHats.mean())
    localAE = np.abs(dsi.trueAlphas.flatten() - dsi.alphaHats.mean(1))
    absErrs["global"].append(globalAE.sum())
    absErrs["local"].append(localAE.sum())
    aucVals["local"].append(correctedAUC(dsi,dsi.alphaHats.mean(1)))
    aucVals["global"].append(correctedAUC(dsi,np.ones(dsi.N)*dsi.globalAlphaHats.mean()))
    # Random Clustering
    means = np.zeros((NIters,dsi.N))
    for iteration in range(NIters):
        wrd = WardClustering(dsi,
                             numbootstraps=dsi.alphaHats.shape[1],
                             randomPairing=True)
        wrd.cluster()
        leafMeans = run(dsi,
                        wrd.clusterAssignment.astype(int),
                        wrd.alphaHatMat,
                        lr=0.01, NIterations=200)
        means[iteration] = leafMeans
    ests = means.mean(0)
    absErrs["forest"].append(np.abs(dsi.trueAlphas.flatten() - ests).sum())
    aucVals["forest"].append(correctedAUC(dsi, ests))
    N += dsi.N
    print("MAE")
    print("local: {:.3f}".format(np.sum(absErrs["local"])/N))
    print("forest: {:.3f}".format(np.sum(absErrs["forest"])/N))
    print("global: {:.3f}".format(np.sum(absErrs["global"])/N))
    print("AUC")
    print("local: {:.3f}".format(np.mean(aucVals["local"])))
    print("forest: {:.3f}".format(np.mean(aucVals["forest"])))
    print("global: {:.3f}".format(np.mean(aucVals["global"])))

MAE
local: 0.124
forest: 0.112
global: 0.215
AUC
local: 0.889
forest: 0.887
global: 0.817


MAE
local: 0.154
forest: 0.149
global: 0.232
AUC
local: 0.804
forest: 0.803
global: 0.753


MAE
local: 0.167
forest: 0.147
global: 0.223
AUC
local: 0.849
forest: 0.848
global: 0.809


MAE
local: 0.162
forest: 0.132
global: 0.230
AUC
local: 0.878
forest: 0.878
global: 0.845


MAE
local: 0.180
forest: 0.153
global: 0.260
AUC
local: 0.851
forest: 0.852
global: 0.818


KeyboardInterrupt: 